In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

from torchvision import transforms
from collections import namedtuple
from torchvision.datasets import ImageFolder

import torchvision.models as models

from sklearn.metrics import classification_report
from torch.nn import functional as F

from torch import nn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Chuẩn bị dữ liệu

In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

#hàm biến đổi dữ liệu 
def prepare_data():
  transform_train = transforms.Compose([
                                  transforms.Resize((256, 256)),                                        #resize images
                                  transforms.RandomCrop(224, padding=4),                 #lấy vùng ngẫu nhiên trong ảnh
                                  transforms.RandomHorizontalFlip(),                             #lat ngang
                                  transforms.ToTensor(),                                                       #convert to Tensor
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])

  transform_test = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(),                           
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  
  path_data_train = "/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/gk4/simple_OCR/data/train"
  path_data_test = "/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/gk4/simple_OCR/data/test"

  trainset = torchvision.datasets.ImageFolder(root=path_data_train, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=path_data_test, transform=transform_test)

  return TrainTest(
      train=trainset,
      test=testset
  )

In [4]:
#hàm load dữ liệu thành các batch để xử lý 
def prepare_loader(datasets):
  batch_size = 8
  trainloader = torch.utils.data.DataLoader(
      datasets.train, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = torch.utils.data.DataLoader(
      datasets.test, batch_size=batch_size, shuffle=False, num_workers=2)
  return TrainTest(
      train=trainloader,
      test=testloader
  )

#Mô hình

In [5]:
class MyNet(nn.Module):
  def __init__(self):
    super().__init__()          #gọi hàm khởi tạo của lớp nn để bắt đầu một modun 
    self.features = models.resnet18(pretrained=True)
    self.features.fc = torch.nn.Linear(512, 4)        #chuyển thành 4 classes 
  
  def forward(self, x):
    out = self.features(x)
    out = F.log_softmax(out, dim=1)
    return out

#Train

In [6]:
def train_epoch(epoch, model, train_loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0  #gía trị hàm loss 
  reporting_step = 8

  #step = 0

  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)
    #step += 1

    #không tích luỹ đạo hàm 
    optimizer.zero_grad()
    outputs = model(images)                 
    loss = loss_func(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    #print(f"Reporting_step:{reporting_step} i:{i}")
    if i % reporting_step == reporting_step - 1:
      print(f"Epoch: {epoch} step: {i} average loss {running_loss / reporting_step:0.4f}")
      running_loss = 0.0

def test_epoch(epoch, model, test_loader, device):
  model.eval()
  y_pred = []
  y_true = []
  
  for i, (images, labels) in enumerate(test_loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, dim=1)
    y_pred += list(predicted.cpu().numpy())
    y_true += list(labels.cpu().numpy())
  return y_pred, y_true

#loss và optimizer 
def get_trainer(model):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  return loss, optimizer

#Hàm main

In [7]:
def main(PATH='/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/gk4/simple_OCR/model_simpleOCR.pth'):
  classes = ['highlands', 'others', 'phuclong', 'starbucks']
  datasets = prepare_data()         #convert dữ liệu

  print("data", len(datasets.train), len(datasets.test))

  loaders =prepare_loader(datasets)                         #đọc dữ liệu vào từng batch một 
  model_simpleOCR = MyNet()

  model_simpleOCR.load_state_dict(torch.load('/content/drive/MyDrive/ki_1_4/Xu_ly_anh/Thay_cuong/gk4/simple_OCR/model_simpleOCR.pth'))

  loss, optimizer = get_trainer(model_simpleOCR)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  number_epoch = 10

  print("number class: ", len(classes))

  model_simpleOCR.to(device)

  for epoch in range(number_epoch):
    print('----------------------------------------------------------')
    train_epoch(epoch, model_simpleOCR, loaders.train, loss, optimizer, device)
    y_pred, y_true = test_epoch(epoch, model_simpleOCR, loaders.test, device)

    print(classification_report(y_true, y_pred, target_names=classes))
    torch.save(model_simpleOCR.state_dict(), PATH)
  
  return model_simpleOCR

model_simpleOCR = main()

data 496 100


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

number class:  4
----------------------------------------------------------
Epoch: 0 step: 7 average loss 0.0982
Epoch: 0 step: 15 average loss 0.0553
Epoch: 0 step: 23 average loss 0.2174
Epoch: 0 step: 31 average loss 0.1145
Epoch: 0 step: 39 average loss 0.0666
Epoch: 0 step: 47 average loss 0.4519
Epoch: 0 step: 55 average loss 0.1023
              precision    recall  f1-score   support

   highlands       1.00      0.97      0.99        40
      others       0.95      1.00      0.97        18
    phuclong       1.00      1.00      1.00        26
   starbucks       1.00      1.00      1.00        16

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100

----------------------------------------------------------
Epoch: 1 step: 7 average loss 0.1559
Epoch: 1 step: 15 average loss 0.0211
Epoch: 1 step: 23 average loss 0.2138
Epoch: 1 step: 31 average loss 0.1391
Epoch: 1 step: